In [201]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
#os.environ['CUDA_LAUNCH_BLOCKING'] = str(1)
#os.environ["TORCH_USE_CUDA_DSA"]= str(0)
device = "cuda"

In [202]:
import copy
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset
import torch.optim as optim
from torch.autograd import Variable
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import math
from collections import OrderedDict
import random

In [232]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(self.input_size, hidden_size, num_layers, bidirectional=False, batch_first=True)




    def forward(self, x):
        out = self.lstm(x)
        return out

In [257]:
model = LSTM(1, 1, 2)

pred, (h,c) = model(torch.randn(5, 1))

print(f"pred: {pred}")
print(f"h: {h}")
print(f"c: {c}")

pred: tensor([[0.1298],
        [0.2080],
        [0.2458],
        [0.2797],
        [0.3005]], grad_fn=<SqueezeBackward1>)
h: tensor([[-0.1508],
        [ 0.3005]], grad_fn=<SqueezeBackward1>)
c: tensor([[-0.8845],
        [ 0.9227]], grad_fn=<SqueezeBackward1>)
